In [3]:
INPUT = """Player 1:
25
37
35
16
9
26
17
5
47
32
11
43
40
15
7
19
36
20
50
3
21
34
44
18
22

Player 2:
12
1
27
41
4
39
13
29
38
2
33
28
10
6
24
31
42
8
23
45
46
48
49
30
14""".split("\n")

In [26]:
def make_list(values):
    head = [None, None]
    tail = head
    for value in values:
        tail[1] = [value, None]
        tail = tail[1]
    return head, tail

def get_list_length(head):
    n = 0
    while head[1]:
        n += 1
        head = head[1]
    return n

CHARS = "".join(chr(ord("A") + n) for n in range(0, 26)) + "".join(chr(ord("a") + n) for n in range(0, 26))

class Player:
    def __init__(self, name, cards):
        self.name = name
        self.head, self.tail = make_list(cards)
        self.card_count = get_list_length(self.head)

    def is_done(self):
        return self.head == self.tail
    
    def draw(self):
        if self.tail is self.head:
            return None
        card = self.head[1]
        self.head[1] = card[1]
        if self.tail is card:
            self.tail = self.head
        self.card_count -= 1
        return card[0]
    
    def add(self, card):
        self.tail[1] = [card, None]
        self.tail = self.tail[1]
        self.card_count += 1
    
    def cards(self):
        cards = []
        card = self.head[1]
        while card:
            cards.append(card[0])
            card = card[1]
        return cards
    
    def score(self):
        cards = self.cards()
        factor = len(cards)
        score = 0
        for card in cards:
            score += card * factor
            factor -= 1
        return score

    def deck_id(self):
        return str(self.name) + "".join(CHARS[c] for c in self.cards())

def play(player1, player2):
    while not player1.is_done() and not player2.is_done():
        card1, card2 = player1.draw(), player2.draw()
        if card1 > card2:
            player1.add(card1)
            player1.add(card2)
            yield f"Player 1 wins the round! {card1} vs {card2}"
        else:
            player2.add(card2)
            player2.add(card1)
            yield f"Player 2 wins the round! {card2} vs {card1}"
        
PLAYER1 = Player("1", (int(s) for s in INPUT[1:26]))
PLAYER2 = Player("2", (int(s) for s in INPUT[28:]))
for i, s in enumerate(play(PLAYER1, PLAYER2)):
    if i > 1000:
        break
    print(i, s)


0 Player 1 wins the round! 25 vs 12
1 Player 1 wins the round! 37 vs 1
2 Player 1 wins the round! 35 vs 27
3 Player 2 wins the round! 41 vs 16
4 Player 1 wins the round! 9 vs 4
5 Player 2 wins the round! 39 vs 26
6 Player 1 wins the round! 17 vs 13
7 Player 2 wins the round! 29 vs 5
8 Player 1 wins the round! 47 vs 38
9 Player 1 wins the round! 32 vs 2
10 Player 2 wins the round! 33 vs 11
11 Player 1 wins the round! 43 vs 28
12 Player 1 wins the round! 40 vs 10
13 Player 1 wins the round! 15 vs 6
14 Player 2 wins the round! 24 vs 7
15 Player 2 wins the round! 31 vs 19
16 Player 2 wins the round! 42 vs 36
17 Player 1 wins the round! 20 vs 8
18 Player 1 wins the round! 50 vs 23
19 Player 2 wins the round! 45 vs 3
20 Player 2 wins the round! 46 vs 21
21 Player 2 wins the round! 48 vs 34
22 Player 2 wins the round! 49 vs 44
23 Player 2 wins the round! 30 vs 18
24 Player 1 wins the round! 22 vs 14
25 Player 2 wins the round! 41 vs 25
26 Player 2 wins the round! 16 vs 12
27 Player 2 wins the

In [20]:
print(PLAYER1.score())

35370


# Part 2

In [50]:
def play2(player1, player2):
    play2.MAX_GAME_NO = 1000
    def recursive_play(player1, player2, game, known_outcomes):
        input_id = f"{player1.deck_id()}:{player2.deck_id()}"
        if input_id in known_outcomes:
            if game <= play2.MAX_GAME_NO:
                print(f"Player {known_outcomes[input_id]} wins game {game}. Known outcome.")            
                play2.MAX_GAME_NO = game
            return player1 if known_outcomes[input_id] == player1.name else player2
        known_decks = set()
        rnd = 0
        while not player1.is_done() and not player2.is_done():
            p1_deck, p2_deck = player1.deck_id(), player2.deck_id()
            if p1_deck in known_decks or p2_deck in known_decks:
                break
            known_decks.add(p1_deck)
            known_decks.add(p2_deck)
            card1, card2 = player1.draw(), player2.draw()
            if card1 <= player1.card_count and card2 <= player2.card_count:
                winner = recursive_play(Player(player1.name,
                                               player1.cards()[:card1]),
                                        Player(player2.name,
                                               player2.cards()[:card2]),
                                        game + 1,
                                        known_outcomes)
                if player1.name == winner.name:
                    winner = player1
                else:
                    winner = player2
                    card1, card2 = card2, card1
            elif card1 > card2:
                winner = player1
            else:
                winner = player2
                card1, card2 = card2, card1

            winner.add(card1)
            winner.add(card2)
            rnd += 1
            if rnd > 100000:
                raise Exception("Too many rounds!!")
                
        winner = player2 if player1.is_done() else player1
        if game < 7:
            print(f"Player {winner.name} wins game {game} after {rnd} rounds!")
        known_outcomes[input_id] = winner.name
        return winner
    
    return recursive_play(player1, player2, 1, {})

PLAYER1 = Player("1", (int(s) for s in INPUT[1:26]))
PLAYER2 = Player("2", (int(s) for s in INPUT[28:]))

WINNER = play2(PLAYER1, PLAYER2)
print(WINNER.score())
print(WINNER.cards())

Player 1 wins game 2 after 66 rounds!
Player 1 wins game 4 after 5 rounds!
Player 1 wins game 3 after 204 rounds!
Player 1 wins game 3 after 11 rounds!
Player 1 wins game 3 after 6 rounds!
Player 1 wins game 4 after 43 rounds!
Player 1 wins game 3 after 71 rounds!
Player 1 wins game 4 after 3 rounds!
Player 1 wins game 3 after 47 rounds!
Player 1 wins game 3 after 50 rounds!
Player 1 wins game 2 after 89 rounds!
Player 1 wins game 2 after 172 rounds!
Player 1 wins game 3 after 37 rounds!
Player 1 wins game 3 after 3 rounds!
Player 1 wins game 4 after 31 rounds!
Player 1 wins game 3 after 50 rounds!
Player 1 wins game 3 after 3 rounds!
Player 1 wins game 3 after 58 rounds!
Player 1 wins game 4 after 18 rounds!
Player 1 wins game 3 after 57 rounds!
Player 1 wins game 3 after 1 rounds!
Player 1 wins game 2 after 162 rounds!
Player 1 wins game 3 after 55 rounds!
Player 1 wins game 3 after 19 rounds!
Player 2 wins game 4 after 1 rounds!
Player 2 wins game 4 after 26 rounds!
Player 2 wins ga

Player 2 wins game 4 after 32 rounds!
Player 2 wins game 4 after 26 rounds!
Player 1 wins game 3 after 73 rounds!
Player 1 wins game 3 after 18 rounds!
Player 1 wins game 3 after 20 rounds!
Player 1 wins game 4 after 17 rounds!
Player 1 wins game 4 after 6 rounds!
Player 1 wins game 4 after 15 rounds!
Player 1 wins game 3 after 103 rounds!
Player 1 wins game 3 after 77 rounds!
Player 2 wins game 4 after 9 rounds!
Player 1 wins game 3 after 81 rounds!
Player 2 wins game 3 after 9 rounds!
Player 1 wins game 3 after 17 rounds!
Player 1 wins game 3 after 71 rounds!
Player 1 wins game 3 after 35 rounds!
Player 1 wins game 3 after 1 rounds!
Player 1 wins game 3 after 12 rounds!
Player 1 wins game 2 after 189 rounds!
Player 1 wins game 2. Known outcome.
Player 2 wins game 3 after 18 rounds!
Player 1 wins game 3 after 34 rounds!
Player 1 wins game 3 after 260 rounds!
Player 1 wins game 3 after 73 rounds!
Player 1 wins game 4 after 27 rounds!
Player 1 wins game 4 after 35 rounds!
Player 2 wins 

Player 1 wins game 4 after 327 rounds!
Player 1 wins game 3 after 73 rounds!
Player 1 wins game 3 after 10 rounds!
Player 1 wins game 4 after 9 rounds!
Player 1 wins game 3 after 33 rounds!
Player 1 wins game 4 after 17 rounds!
Player 1 wins game 3 after 209 rounds!
Player 1 wins game 3 after 17 rounds!
Player 1 wins game 3 after 13 rounds!
Player 1 wins game 4 after 3 rounds!
Player 1 wins game 4 after 7 rounds!
Player 1 wins game 4 after 10 rounds!
Player 1 wins game 3 after 66 rounds!
Player 1 wins game 3 after 70 rounds!
Player 1 wins game 3 after 58 rounds!
Player 1 wins game 3 after 20 rounds!
Player 1 wins game 5 after 18 rounds!
Player 1 wins game 4 after 85 rounds!
Player 1 wins game 4 after 42 rounds!
Player 1 wins game 4 after 6 rounds!
Player 1 wins game 4 after 11 rounds!
Player 1 wins game 3 after 87 rounds!
Player 2 wins game 3 after 17 rounds!
Player 2 wins game 3 after 8 rounds!
Player 1 wins game 3 after 20 rounds!
Player 1 wins game 3 after 30 rounds!
Player 1 wins g

Player 1 wins game 4 after 54 rounds!
Player 1 wins game 4 after 2 rounds!
Player 1 wins game 4 after 6 rounds!
Player 1 wins game 3 after 245 rounds!
Player 2 wins game 4 after 3 rounds!
Player 1 wins game 3 after 129 rounds!
Player 1 wins game 3 after 75 rounds!
Player 1 wins game 3 after 20 rounds!
Player 1 wins game 4 after 6 rounds!
Player 1 wins game 4 after 64 rounds!
Player 1 wins game 5 after 1 rounds!
Player 1 wins game 5 after 2 rounds!
Player 1 wins game 4 after 12 rounds!
Player 1 wins game 4 after 1 rounds!
Player 1 wins game 4 after 12 rounds!
Player 1 wins game 4 after 8 rounds!
Player 1 wins game 4 after 1 rounds!
Player 1 wins game 3 after 74 rounds!
Player 1 wins game 6 after 8 rounds!
Player 1 wins game 6 after 1 rounds!
Player 1 wins game 6 after 2 rounds!
Player 1 wins game 6 after 159 rounds!
Player 1 wins game 6 after 14 rounds!
Player 1 wins game 6 after 7 rounds!
Player 1 wins game 6 after 12 rounds!
Player 1 wins game 6 after 11 rounds!
Player 1 wins game 5 a

Player 1 wins game 5 after 229 rounds!
Player 1 wins game 6 after 3 rounds!
Player 1 wins game 5 after 73 rounds!
Player 1 wins game 6 after 17 rounds!
Player 1 wins game 5 after 36 rounds!
Player 1 wins game 5 after 52 rounds!
Player 1 wins game 5 after 1 rounds!
Player 1 wins game 5 after 22 rounds!
Player 1 wins game 6 after 3 rounds!
Player 1 wins game 5 after 15 rounds!
Player 2 wins game 5 after 48 rounds!
Player 1 wins game 6 after 3 rounds!
Player 1 wins game 6 after 22 rounds!
Player 1 wins game 5 after 139 rounds!
Player 1 wins game 5 after 22 rounds!
Player 1 wins game 6 after 25 rounds!
Player 1 wins game 5 after 63 rounds!
Player 1 wins game 5 after 1 rounds!
Player 1 wins game 5 after 24 rounds!
Player 1 wins game 5 after 16 rounds!
Player 1 wins game 5 after 1 rounds!
Player 1 wins game 4 after 146 rounds!
Player 2 wins game 4 after 3 rounds!
Player 2 wins game 5 after 2 rounds!
Player 2 wins game 5 after 9 rounds!
Player 1 wins game 4 after 55 rounds!
Player 2 wins game

...

Player 1 wins game 5 after 13 rounds!
Player 1 wins game 5 after 5 rounds!
Player 1 wins game 5 after 80 rounds!
Player 1 wins game 5 after 41 rounds!
Player 1 wins game 6 after 1 rounds!
Player 1 wins game 5 after 26 rounds!
Player 1 wins game 5 after 1 rounds!
Player 1 wins game 5 after 7 rounds!
Player 1 wins game 4 after 311 rounds!
Player 2 wins game 4 after 7 rounds!
Player 1 wins game 3 after 341 rounds!
Player 2 wins game 4 after 8 rounds!
Player 1 wins game 4 after 20 rounds!
Player 1 wins game 4 after 51 rounds!
Player 1 wins game 4 after 33 rounds!
Player 1 wins game 5 after 2 rounds!
Player 1 wins game 4 after 27 rounds!
Player 1 wins game 4 after 22 rounds!
Player 1 wins game 5 after 2 rounds!
Player 1 wins game 4 after 47 rounds!
Player 2 wins game 5 after 19 rounds!
Player 1 wins game 4 after 66 rounds!
Player 2 wins game 5 after 5 rounds!
Player 1 wins game 5 after 7 rounds!
Player 1 wins game 5 after 59 rounds!
Player 1 wins game 5 after 37 rounds!
Player 1 wins game 6

Player 1 wins game 2 after 87 rounds!
Player 1 wins game 4 after 2 rounds!
Player 1 wins game 3 after 9 rounds!
Player 1 wins game 2 after 429 rounds!
Player 1 wins game 3 after 13 rounds!
Player 1 wins game 3 after 2 rounds!
Player 1 wins game 3 after 27 rounds!
Player 2 wins game 3 after 15 rounds!
Player 2 wins game 3 after 79 rounds!
Player 1 wins game 3 after 39 rounds!
Player 1 wins game 3 after 28 rounds!
Player 2 wins game 3 after 15 rounds!
Player 1 wins game 3 after 42 rounds!
Player 2 wins game 3 after 31 rounds!
Player 2 wins game 3 after 67 rounds!
Player 2 wins game 4 after 3 rounds!
Player 2 wins game 3 after 93 rounds!
Player 1 wins game 3 after 25 rounds!
Player 2 wins game 3 after 8 rounds!
Player 2 wins game 3 after 3 rounds!
Player 2 wins game 2 after 400 rounds!
Player 1 wins game 2 after 15 rounds!
Player 1 wins game 3 after 2 rounds!
Player 1 wins game 3 after 244 rounds!
Player 1 wins game 2 after 79 rounds!
Player 1 wins game 2. Known outcome.
Player 1 wins gam

Player 1 wins game 3 after 699 rounds!
Player 2 wins game 5 after 6 rounds!
Player 2 wins game 4 after 54 rounds!
Player 1 wins game 4 after 2 rounds!
Player 2 wins game 4 after 10 rounds!
Player 1 wins game 4 after 52 rounds!
Player 2 wins game 4 after 7 rounds!
Player 2 wins game 4 after 6 rounds!
Player 2 wins game 4 after 3 rounds!
Player 2 wins game 3 after 117 rounds!
Player 1 wins game 4 after 1 rounds!
Player 2 wins game 3 after 45 rounds!
Player 2 wins game 5 after 3 rounds!
Player 2 wins game 4 after 10 rounds!
Player 1 wins game 4 after 4 rounds!
Player 1 wins game 4 after 56 rounds!
Player 1 wins game 4 after 47 rounds!
Player 1 wins game 5 after 19 rounds!
Player 1 wins game 4 after 70 rounds!
Player 1 wins game 4 after 32 rounds!
Player 1 wins game 4 after 26 rounds!
Player 1 wins game 4 after 36 rounds!
Player 1 wins game 3 after 107 rounds!
Player 1 wins game 3 after 56 rounds!
Player 1 wins game 3 after 12 rounds!
Player 1 wins game 3 after 16 rounds!
Player 2 wins gam

Player 1 wins game 4 after 91 rounds!
Player 1 wins game 5 after 11 rounds!
Player 1 wins game 4 after 18 rounds!
Player 2 wins game 4 after 2 rounds!
Player 1 wins game 5 after 6 rounds!
Player 1 wins game 5 after 14 rounds!
Player 1 wins game 4 after 63 rounds!
Player 1 wins game 4 after 31 rounds!
Player 2 wins game 4 after 4 rounds!
Player 1 wins game 4 after 7 rounds!
Player 1 wins game 4 after 37 rounds!
Player 1 wins game 5 after 1 rounds!
Player 1 wins game 5 after 3 rounds!
Player 1 wins game 6 after 1 rounds!
Player 1 wins game 5 after 28 rounds!
Player 1 wins game 6 after 13 rounds!
Player 1 wins game 6 after 5 rounds!
Player 1 wins game 5 after 82 rounds!
Player 1 wins game 5 after 4 rounds!
Player 1 wins game 4 after 108 rounds!
Player 1 wins game 4 after 1 rounds!
Player 1 wins game 4 after 3 rounds!
Player 1 wins game 5 after 2 rounds!
Player 1 wins game 4 after 18 rounds!
Player 1 wins game 4 after 2 rounds!
Player 1 wins game 3 after 170 rounds!
Player 1 wins game 5 af

Player 1 wins game 3 after 85 rounds!
Player 1 wins game 6 after 19 rounds!
Player 1 wins game 6 after 3 rounds!
Player 1 wins game 5 after 28 rounds!
Player 1 wins game 5 after 19 rounds!
Player 1 wins game 4 after 43 rounds!
Player 1 wins game 3 after 108 rounds!
Player 1 wins game 2 after 164 rounds!
Player 2 wins game 2. Known outcome.
Player 1 wins game 2. Known outcome.
Player 1 wins game 2 after 26 rounds!
Player 1 wins game 3 after 6 rounds!
Player 1 wins game 2 after 17 rounds!
Player 1 wins game 2. Known outcome.
Player 1 wins game 2 after 2 rounds!
Player 1 wins game 2 after 1 rounds!
Player 1 wins game 3 after 17 rounds!
Player 1 wins game 4 after 6 rounds!
Player 1 wins game 3 after 19 rounds!
Player 1 wins game 3 after 13 rounds!
Player 1 wins game 3 after 6 rounds!
Player 1 wins game 3 after 7 rounds!
Player 1 wins game 2 after 54 rounds!
Player 1 wins game 2. Known outcome.
Player 1 wins game 2 after 19 rounds!
Player 1 wins game 2 after 7 rounds!
Player 1 wins game 2 a

In [46]:
WINNER.score()

18054

In [29]:
WINNER.deck_id()

'1lBkbiZvruqsfjeygxVwdoLthmYpcnXaS'

In [31]:
PLAYER1 = Player("1", (int(s) for s in INPUT[1:26]))
PLAYER2 = Player("2", (int(s) for s in INPUT[28:]))

PLAYER2.deck_id()

'2MBbpEnNdmChcKGYfqIXtuwxeO'